## Outer version


In [8]:
# -*- coding: utf-8 -*-
"""
IEOR4004 Project 1 — Part 2 solver (robust version)
Requirements: gurobipy, pandas, numpy
Place CSVs in same folder or update the file paths below.
"""

import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from itertools import combinations
from pathlib import Path

# ------------------------------
# 1. Data Paths (edit if needed)
# ------------------------------


POP_FILE   = Path("Project 1 Data- version 0.0- Yue Chen/population_calculated.csv")
CLASS_FILE = Path("Project 1 Data- version 0.0- Yue Chen/demand_classification.csv")
FAC_FILE   = Path("Project 1 Data- version 0.0- Yue Chen/child_care_regulated_cleaned.csv")
SITE_FILE  = Path("Project 1 Data- version 0.0- Yue Chen/potential_locations.csv")

# ------------------------------
# 2. Global Parameters
# ------------------------------
BETA  = 100.0          # Equipment cost per 0–5 slot
DELTA = 0.06           # Minimum distance (miles) for conflict
Q05_IMPUTE_RATIO = 1 # Default ratio if q_0_5 missing

# ------------------------------
# 3. Default Facility Sizes
# ------------------------------
DEFAULT_SIZES = pd.DataFrame({
    "k": ["S", "M", "L"],
    "cap_k": [100, 200, 400],
    "cap_k_0_5": [50, 100, 200],
    "cost_k": [65000, 95000, 115000],
})

In [9]:
# ------------------------------
# 4. Helper functions
# ------------------------------
def pad_zip(z):
    """Ensure ZIP codes are five digits (e.g., 123 -> 00123)."""
    if pd.isna(z):
        return ""
    s = str(z).strip()
    if s.endswith(".0"):
        s = s[:-2]
    return s.zfill(5)

def haversine(lat1, lon1, lat2, lon2):
    """Compute great-circle distance (miles) between two (lat, lon) points."""
    if any(pd.isna(v) for v in (lat1, lon1, lat2, lon2)):
        return np.inf
    R = 3958.8  # Earth radius in miles
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    a = np.sin((lat2 - lat1) / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin((lon2 - lon1) / 2.0)**2
    return 2.0 * R * np.arcsin(np.sqrt(a))


In [10]:
# ------------------------------
# 5. Data Loading and cleaning
# ------------------------------
def load_data():
    """Load and preprocess population, classification, facilities, and site data."""
    # --- Load population ---
    pop = pd.read_csv(POP_FILE)
    pop["zip_code_cleaned"] = pop["zip_code_cleaned"].apply(pad_zip)
    pop.rename(columns={"Population_0_12": "pop_0_12", "Population_0_5": "pop_0_5"}, inplace=True)
    pop["pop_0_12"] = pd.to_numeric(pop.get("pop_0_12", 0), errors="coerce").fillna(0)
    pop["pop_0_5"]  = pd.to_numeric(pop.get("pop_0_5", 0), errors="coerce").fillna(0)

    # --- Load demand classification ---
    cls = pd.read_csv(CLASS_FILE)
    cls["zip_code_cleaned"] = cls["zip_code_cleaned"].apply(pad_zip)
    cls.rename(columns={"Demand_Classification": "class"}, inplace=True)
    cls["class"] = cls["class"].astype(str).str.strip().str.lower()
    cls["t_i"] = np.where(cls["class"].str.contains("high", na=False), 0.5, 1/3)

    # --- Outer join instead of intersection ---
    areas = pop.merge(
        cls[["zip_code_cleaned", "t_i"]],
        on="zip_code_cleaned",
        how="outer",
        validate="one_to_one"
    )

    # --- Fill missing data safely ---
    areas["pop_0_12"] = areas["pop_0_12"].fillna(0)
    areas["pop_0_5"] = areas["pop_0_5"].fillna(0)
    areas["t_i"] = areas["t_i"].fillna(1/3)

    # --- Enforce correct types ---
    areas["pop_0_12"] = areas["pop_0_12"].astype(float)
    areas["pop_0_5"] = areas["pop_0_5"].astype(float)
    areas["t_i"] = areas["t_i"].astype(float)

    print(f"[load_data] ZIP union: {len(areas)} total (pop={len(pop)}, cls={len(cls)})")

    # --- Facility data ---
    fac = pd.read_csv(FAC_FILE)
    fac["zip_code_cleaned"] = fac["zip_code_cleaned"].apply(pad_zip)
    if "total_capacity" in fac.columns:
        fac.rename(columns={"total_capacity": "n_f"}, inplace=True)
    if "latitude" in fac.columns:
        fac.rename(columns={"latitude": "lat"}, inplace=True)
    if "longitude" in fac.columns:
        fac.rename(columns={"longitude": "lon"}, inplace=True)

    # compute or impute q_0_5
    if "q_0_5" not in fac.columns:
        if all(col in fac.columns for col in ["infant_capacity", "toddler_capacity", "preschool_capacity"]):
            fac["q_0_5"] = fac["infant_capacity"].fillna(0) + fac["toddler_capacity"].fillna(0) + fac["preschool_capacity"].fillna(0)
        else:
            fac["n_f"] = pd.to_numeric(fac.get("n_f", 0), errors="coerce").fillna(0)
            fac["q_0_5"] = (fac["n_f"] * Q05_IMPUTE_RATIO).round(0)

    fac = fac[["facility_id", "zip_code_cleaned", "n_f", "q_0_5", "lat", "lon"]].copy()
    fac["n_f"] = pd.to_numeric(fac["n_f"], errors="coerce").fillna(0)
    fac["q_0_5"] = pd.to_numeric(fac["q_0_5"], errors="coerce").fillna(0)

    before_fac_count = len(fac)
    fac = fac[fac["n_f"] > 0].copy()
    after_fac_count = len(fac)

    # --- Site data ---
    site = pd.read_csv(SITE_FILE)
    site.rename(columns={"zipcode": "zip_code_cleaned", "latitude": "lat", "longitude": "lon"}, inplace=True)
    site["zip_code_cleaned"] = site["zip_code_cleaned"].apply(pad_zip)
    site["site_id"] = np.arange(1, len(site) + 1)
    site = site[["site_id", "zip_code_cleaned", "lat", "lon"]].copy()

    sizes = DEFAULT_SIZES.copy()

    # --- Drop known infeasible ZIPs ---
    drop_zips = ["11219"]
    areas = areas[~areas["zip_code_cleaned"].isin(drop_zips)].copy()
    print(f"[load_data] Dropped infeasible ZIPs: {drop_zips}")

    print(f"[load_data] areas: {len(areas)}, facilities (before filter): {before_fac_count}, (after filter): {after_fac_count}, sites: {len(site)}")
    print("[load_data] sample areas head:\n", areas.head().to_string(index=False))

    return areas, fac, site, sizes


In [11]:
# ------------------------------
# 6. Spatial feasibility and conflicts
# ------------------------------
def infer_sites(sites, facs, delta=DELTA):
    """Mark candidate sites as feasible (1) or infeasible (0) if within DELTA miles of existing facility."""
    sites = sites.copy()
    sites["allowed"] = 1
    if len(facs) == 0:
        sites["allowed"] = 1
        return sites
    for i, s in sites.iterrows():
        # check existing facilities in same zip first (faster)
        zf = facs[facs["zip_code_cleaned"] == s["zip_code_cleaned"]]
        infeasible = False
        for _, f in zf.iterrows():
            if haversine(s["lat"], s["lon"], f["lat"], f["lon"]) < delta:
                infeasible = True
                break
        sites.loc[i, "allowed"] = 0 if infeasible else 1
    return sites

def conflict_pairs(sites, delta=DELTA):
    """Find pairs of sites that are closer than delta miles (we restrict to sites within same zip for efficiency)."""
    pairs = []
    if len(sites) == 0:
        return pairs
    for zip_i, grp in sites.groupby("zip_code_cleaned"):
        idxs = list(grp.index)
        if len(idxs) < 2:
            continue
        for i, j in combinations(idxs, 2):
            if haversine(grp.loc[i, "lat"], grp.loc[i, "lon"], grp.loc[j, "lat"], grp.loc[j, "lon"]) < delta:
                pairs.append((grp.loc[i, "site_id"], grp.loc[j, "site_id"]))
    print(f"[conflict_pairs] conflict pairs found: {len(pairs)}")
    return pairs


In [12]:
# ------------------------------
# 7. Cost segments (protect divide-by-zero)
# ------------------------------
def cost_segments(n):
    """Return unit costs for three expansion segments based on current capacity n.
       If n is <= 0, return very large unit costs (should have been filtered out earlier)."""
    if n <= 0 or pd.isna(n):
        # safe fallback: very large per-slot cost to discourage selecting this facility
        large = 1e6
        return (large, large, large)
    return ((20000 + 200.0 * n) / n,
            (20000 + 400.0 * n) / n,
            (20000 + 1000.0 * n) / n)

In [ ]:
# ------------------------------
# 8. Build model
# ------------------------------
def build_model(areas, facs, sites, sizes, pairs, time_limit=120, mip_gap=0.01):
    """Create Gurobi model following the problem formulation (robust to empty sets)."""
    I = list(areas["zip_code_cleaned"])
    F = list(facs["facility_id"])
    S = list(sites["site_id"])
    K = list(sizes["k"])

    # parameter maps (use dict.get with default to avoid KeyError)
    nf   = dict(zip(facs.facility_id, facs.n_f))
    q05  = dict(zip(facs.facility_id, facs.q_0_5))
    zip_f = dict(zip(facs.facility_id, facs.zip_code_cleaned))
    zip_s = dict(zip(sites.site_id, sites.zip_code_cleaned))
    ti   = dict(zip(areas.zip_code_cleaned, areas.t_i))
    Pi   = dict(zip(areas.zip_code_cleaned, areas.pop_0_12))
    Pi05 = dict(zip(areas.zip_code_cleaned, areas.pop_0_5))
    cap  = dict(zip(sizes.k, sizes.cap_k))
    cap05= dict(zip(sizes.k, sizes.cap_k_0_5))
    cost = dict(zip(sizes.k, sizes.cost_k))
    allow= dict(zip(sites.site_id, sites.allowed if "allowed" in sites.columns else [1]*len(sites)))

    # segment unit costs
    c1 = {f: cost_segments(nf.get(f, 0))[0] for f in F}
    c2 = {f: cost_segments(nf.get(f, 0))[1] for f in F}
    c3 = {f: cost_segments(nf.get(f, 0))[2] for f in F}

    # create model
    m = gp.Model("IEOR4004_Project1_Part2_Gurobi")
    m.Params.OutputFlag = 1
    m.setParam("MIPGap", mip_gap)
    m.setParam("TimeLimit", time_limit)

    # variables: handle empty sets gracefully
    x1 = m.addVars(F, vtype=GRB.CONTINUOUS, name="x_f1") if F else {}
    x2 = m.addVars(F, vtype=GRB.CONTINUOUS, name="x_f2") if F else {}
    x3 = m.addVars(F, vtype=GRB.CONTINUOUS, name="x_f3") if F else {}
    # all y's are continuous in [0,1] (continuous piecewise expansion)
    y1 = m.addVars(F, lb=0, ub=1, name="y_f1") if F else {}
    y2 = m.addVars(F, lb=0, ub=1, name="y_f2") if F else {}
    y3 = m.addVars(F, lb=0, ub=1, name="y_f3") if F else {}
    # removed binary b2, b3
    xF = m.addVars(F, vtype=GRB.CONTINUOUS, name="x_f") if F else {}
    z05 = m.addVars(F, vtype=GRB.CONTINUOUS, name="z_f05") if F else {}

    y_sk   = m.addVars(S, K, vtype=GRB.BINARY, name="y_sk") if S and K else {}
    u_sk05 = m.addVars(S, K, vtype=GRB.CONTINUOUS, name="u_sk05") if S and K else {}

    # objective
    obj_expansion = gp.quicksum(c1[f] * x1[f] + c2[f] * x2[f] + c3[f] * x3[f] for f in F) if F else 0
    obj_newbuild  = gp.quicksum(cost[k] * y_sk[s, k] for s in S for k in K) if S and K else 0
    obj_eqpt      = BETA * (gp.quicksum(z05[f] for f in F) + gp.quicksum(u_sk05[s, k] for s in S for k in K)) if ((F) or (S and K)) else 0
    m.setObjective(obj_expansion + obj_newbuild + obj_eqpt, GRB.MINIMIZE)

    # constraints
    # (a) piecewise expansion (only if F non-empty)
    for f in F:
        n = float(nf.get(f, 0))
        # segments are defined as fractions of n
        m.addConstr(x1[f] == 0.10 * n * y1[f], name=f"x1_def_{f}")
        m.addConstr(x2[f] == 0.05 * n * y2[f], name=f"x2_def_{f}")
        m.addConstr(x3[f] == 0.05 * n * y3[f], name=f"x3_def_{f}")
        # enforce ordered activation across continuous y's
        m.addConstr(y2[f] <= y1[f], name=f"y2_le_y1_{f}")
        m.addConstr(y3[f] <= y2[f], name=f"y3_le_y2_{f}")
        m.addConstr(xF[f] == x1[f] + x2[f] + x3[f], name=f"xF_def_{f}")
        m.addConstr(xF[f] <= 0.20 * n, name=f"xF_ub_{f}")
        m.addConstr(z05[f] <= xF[f], name=f"z05_le_xF_{f}")
 
    """ # (a) simplified expansion constraint (max 20% of current capacity)
    for f in F:
        n = float(nf.get(f, 0))
        
        # ensure non-negative expansion variable
        m.addConstr(xF[f] >= 0, name=f"xF_nonneg_{f}")
        
        # total expansion limited to 20% of existing capacity
        m.addConstr(xF[f] <= 0.20 * n, name=f"xF_ub_{f}")
        
        # link expansion for 0–5 slots if applicable
        # (assuming z05[f] is the expansion allocated to children 0–5)
        m.addConstr(z05[f] <= xF[f], name=f"z05_le_xF_{f}") """

    
    # (b) new site feasibility and 0-5 capacity
    for s in S:
        if allow.get(s, 1) == 0:
            # not allowed to build at this site
            m.addConstr(gp.quicksum(y_sk[s, k] for k in K) == 0, name=f"site_block_{s}")
        else:
            m.addConstr(gp.quicksum(y_sk[s, k] for k in K) <= 1, name=f"site_one_size_{s}")
        for k in K:
            m.addConstr(u_sk05[s, k] <= cap05[k] * y_sk[s, k], name=f"u_cap05_{s}_{k}")

    # (c) area total capacity coverage
    for i in I:
        # existing + expansions in area i
        existing = gp.quicksum(nf[f] + xF[f] for f in F if zip_f.get(f, "") == i) if F else 0
        newcap   = gp.quicksum(cap[k] * y_sk[s, k] for s in S if zip_s.get(s, "") == i for k in K) if (S and K) else 0
        rhs = ti.get(i, 1/3) * Pi.get(i, 0)
        # ensure rhs is numeric
        if pd.isna(rhs):
            rhs = 0
        m.addConstr(existing + newcap >= rhs, name=f"area_cover_{i}")

    # (d) area 0-5 coverage
    for i in I:
        exist05 = gp.quicksum(q05[f] + z05[f] for f in F if zip_f.get(f, "") == i) if F else 0
        new05   = gp.quicksum(u_sk05[s, k] for s in S if zip_s.get(s, "") == i for k in K) if (S and K) else 0
        rhs05 = (2.0/3.0) * Pi05.get(i, 0)
        if pd.isna(rhs05):
            rhs05 = 0
        m.addConstr(exist05 + new05 >= rhs05, name=f"area_05_cover_{i}")

    # (e) site conflict pairs (distance)
    for (s1, s2) in pairs:
        for k1 in K:
            for k2 in K:
                m.addConstr(y_sk[s1, k1] + y_sk[s2, k2] <= 1, name=f"conf_{s1}_{s2}_{k1}_{k2}")

    # done building
    return m, (xF, z05, y_sk, u_sk05)


In [14]:
# ------------------------------
# 9. Solve and print results + IIS analysis
# ------------------------------
def main():
    import pandas as pd
    import numpy as np
    import gurobipy as gp
    from gurobipy import GRB
    from itertools import combinations
    from pathlib import Path

    areas, facs, sites, sizes = load_data()

    # quick sanity checks
    print("[main] sample areas head:\n", areas.head().to_string(index=False))
    print("[main] sample facilities head:\n", facs.head().to_string(index=False))
    print("[main] sample sites head:\n", sites.head().to_string(index=False))

    sites = infer_sites(sites, facs)
    pairs = conflict_pairs(sites)

    print(f"[main] after infer_sites: allowed sites = {sites['allowed'].sum()} / {len(sites)}")

    # build and solve
    m, (xF, z05, y_sk, u_sk05) = build_model(
        areas, facs, sites, sizes, pairs,
        time_limit=120, mip_gap=0.01
    )

    print("[main] Starting optimization...")
    m.optimize()

    status = m.status
    print("[main] Solver status code:", status)

    if status in (GRB.Status.OPTIMAL, GRB.Status.TIME_LIMIT) and m.SolCount > 0:
        print("------ SOLUTION SUMMARY ------")
        print("Objective (Total Cost): ${:,.0f}".format(m.objVal))

        # --- selected new sites ---
        new_sites = []
        cap   = dict(zip(sizes.k, sizes.cap_k))
        cap05 = dict(zip(sizes.k, sizes.cap_k_0_5))
        zip_f = dict(zip(facs.facility_id, facs.zip_code_cleaned))
        zip_s = dict(zip(sites.site_id, sites.zip_code_cleaned))

        for (s, k) in list(y_sk.keys()):
            if y_sk[s, k].X > 0.5:
                new_sites.append({
                    "site_id": s,
                    "zip": zip_s.get(s, ""),
                    "size": k,
                    "cap_total": cap[k],
                    "cap_0_5": u_sk05[s, k].X
                })

        new_sites = sorted(new_sites, key=lambda r: (r["zip"], r["site_id"]))
        print("\n--- New Facilities Selected ---")
        if not new_sites:
            print("  (none)")
        else:
            for r in new_sites:
                print(f"  Site {r['site_id']} | ZIP {r['zip']} | Size {r['size']} "
                      f"| Total cap {r['cap_total']} | 0–5 slots {r['cap_0_5']:.1f}")

        # --- expansions ---
        exp_list = []
        for f in xF.keys():
            xf = xF[f].X
            zf5 = z05[f].X
            if xf > 1e-6 or zf5 > 1e-6:
                exp_list.append({
                    "facility_id": f,
                    "zip": zip_f.get(f, ""),
                    "expand": xf,
                    "add_0_5": zf5
                })

        exp_list = sorted(exp_list, key=lambda r: (r["zip"], r["facility_id"]))
        print("\n--- Facility Expansions ---")
        if not exp_list:
            print("  (none)")
        else:
            for r in exp_list:
                print(f"  Fac {r['facility_id']} | ZIP {r['zip']} | +{r['expand']:.1f} slots | 0–5 +{r['add_0_5']:.1f}")

        # --- totals ---
        total_new_cap = sum(r["cap_total"] for r in new_sites)
        total_new_05  = sum(r["cap_0_5"] for r in new_sites)
        total_exp_cap = sum(r["expand"] for r in exp_list)
        total_exp_05  = sum(r["add_0_5"] for r in exp_list)

        print("\n--- Totals ---")
        print(f"  New facilities: {len(new_sites)} | Total new capacity: {total_new_cap:.0f} | New 0–5 slots: {total_new_05:.1f}")
        print(f"  Expanded facilities: {len(exp_list)} | Total expansion: {total_exp_cap:.1f} | 0–5 from expansion: {total_exp_05:.1f}")
        print("-------------------------------")

    else:
        print("No feasible solution found or no solution returned. Solver status:", status, "SolCount:", m.SolCount)

        # ------------------------------
        # IIS Debugging (Automatic Analysis)
        # ------------------------------
        if status in [GRB.Status.INFEASIBLE, GRB.Status.INF_OR_UNBD]:
            print("\n[debug] Model infeasible; computing IIS...")

            m_feas = m.copy()
            m_feas.computeIIS()

            infeas_constrs = [c.constrName for c in m_feas.getConstrs() if c.IISConstr]
            print(f"[debug] {len(infeas_constrs)} infeasible constraints found.")

            # Try to extract ZIP codes from area constraints
            zip_list = []
            for c in infeas_constrs:
                if c.startswith("area_cover_") or c.startswith("area_05_cover_"):
                    parts = c.split("_")
                    zip_code = parts[-1]
                    zip_list.append(zip_code)

            if zip_list:
                print(f"[debug] Found {len(zip_list)} ZIPs causing infeasibility.")
                df_zip = pd.DataFrame({"infeasible_zip": zip_list})
                df_zip.to_csv("infeasible_zips_full.csv", index=False)
                print("[debug] Saved infeasible ZIPs to infeasible_zips.csv")

            else:
                print("[debug] No ZIP-specific infeasible constraints found.")

            # Export IIS report
            m_feas.write("model.ilp")
            m_feas.write("model.ilp.gz")
            m_feas.write("model.ilp.json")
            print("[debug] IIS written to 'model.ilp.*' files for inspection.")

        else:
            print("[debug] Model is not infeasible according to Gurobi status.")


if __name__ == "__main__":
    main()

[load_data] ZIP union: 1805 total (pop=1646, cls=1534)
[load_data] Dropped infeasible ZIPs: ['11219']
[load_data] areas: 1804, facilities (before filter): 15014, (after filter): 15013, sites: 215400
[load_data] sample areas head:
 zip_code_cleaned  pop_0_5  pop_0_12      t_i
           06390      0.0       4.0 0.333333
           10001    901.0    2093.0 0.333333
           10002   2751.0    7107.0 0.500000
           10003   1647.0    3046.0 0.333333
           10004    469.0     712.0 0.333333
[main] sample areas head:
 zip_code_cleaned  pop_0_5  pop_0_12      t_i
           06390      0.0       4.0 0.333333
           10001    901.0    2093.0 0.333333
           10002   2751.0    7107.0 0.500000
           10003   1647.0    3046.0 0.333333
           10004    469.0     712.0 0.333333
[main] sample facilities head:
  facility_id zip_code_cleaned  n_f  q_0_5       lat        lon
       40163            10960  120    120 41.089425 -73.920413
       41016            12077   75      0 42